In [9]:
from google.colab import drive

# 1. Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# squeezenet model

In [10]:
!pip install ultralytics
import cv2
import torch
import torch.nn as nn
import gradio as gr
import albumentations as A
import numpy as np
import os
from ultralytics import YOLO
from torchvision import models, transforms
from PIL import Image
import torch.nn.functional as F

# --- STEP 1: CONFIGURATION ---
# Replace with the local path to your PLC-SN .pth file
MODEL_PATH = '/content/drive/MyDrive/best_pepper_squeezenet.pth'
# --- STEP 2: DEFINE 16 TTA TECHNIQUES ---
tta_transforms = {
    "Original": None,
    "Horizontal Flip": A.HorizontalFlip(p=1),
    "Vertical Flip": A.VerticalFlip(p=1),
    "Rotate 15°": A.Rotate(limit=(15, 15), p=1),
    "Rotate -15°": A.Rotate(limit=(-15, -15), p=1),
    "Brightness +": A.RandomBrightnessContrast(brightness_limit=(0.2, 0.2), contrast_limit=0, p=1),
    "Brightness -": A.RandomBrightnessContrast(brightness_limit=(-0.2, -0.2), contrast_limit=0, p=1),
    "Contrast +": A.RandomBrightnessContrast(brightness_limit=0, contrast_limit=(0.2, 0.2), p=1),
    "Gaussian Noise": A.GaussNoise(var_limit=(10, 50), p=1),
    "Hue Shift": A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=0, val_shift_limit=0, p=1),
    "Saturation +": A.HueSaturationValue(hue_shift_limit=0, sat_shift_limit=30, val_shift_limit=0, p=1),
    "GrayScale": A.ToGray(p=1),
    "Blur": A.Blur(blur_limit=3, p=1),
    "Sharpen": A.Sharpen(p=1),
    "Posterize": A.Posterize(p=1),
    "Solarize": A.Solarize(p=1)
}

# IMPORTANT: PLC-SN is trained on 500x500 images
final_preprocess = transforms.Compose([
    transforms.Resize((500, 500)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# --- STEP 3: SQUEEZENET MODEL LOADING ---
def load_pepper_model(path):
    # Initialize SqueezeNet 1.1
    model = models.squeezenet1_1(weights=None)
    
    # Modify the classifier to match your 2-class training
    # SqueezeNet uses Conv2d in its classifier block instead of Linear
    model.classifier[1] = nn.Conv2d(512, 2, kernel_size=(1,1))
    model.num_classes = 2
    
    if not os.path.exists(path):
        raise FileNotFoundError(f"Model file not found at: {path}")
        
    state_dict = torch.load(path, map_location=torch.device('cpu'))
    model.load_state_dict(state_dict)
    model.eval()
    return model

detector = YOLO('yolov8n.pt') 
classifier = load_pepper_model(MODEL_PATH)
class_names = ['HEALTHY', 'UNHEALTHY']

# --- STEP 4: PREDICTION LOGIC ---
def predict_pepper(input_img):
    if input_img is None: return None, None, None
    
    img_rgb = np.array(input_img)
    img_display = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2BGR)
    
    # YOLO detection for leaf localization
    results = detector(img_rgb, conf=0.25)
    detections = results[0].boxes if len(results) > 0 else []
    
    final_probs = torch.zeros(2)
    gallery_data = []

    if len(detections) > 0:
        box = detections[0]
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        leaf_crop = img_rgb[y1:y2, x1:x2]
        
        # Run SqueezeNet on all 16 views
        final_probs, gallery_data = run_tta_with_individual_labels(leaf_crop)
        
        label_idx = torch.argmax(final_probs).item()
        color = (0, 255, 0) if label_idx == 0 else (0, 0, 255)
        cv2.rectangle(img_display, (x1, y1), (x2, y2), color, 6)
        cv2.putText(img_display, f"Consensus: {class_names[label_idx]}", (x1, y1-15), 0, 1.2, color, 3)
    else:
        # Fallback to whole image analysis
        final_probs, gallery_data = run_tta_with_individual_labels(img_rgb)

    confidences = {class_names[i]: float(final_probs[i]) for i in range(2)}
    return cv2.cvtColor(img_display, cv2.COLOR_BGR2RGB), confidences, gallery_data

def run_tta_with_individual_labels(image_np):
    all_outputs = []
    gallery_list = []
    
    for name, aug in tta_transforms.items():
        aug_img = aug(image=image_np)['image'] if aug else image_np
        
        pil_img = Image.fromarray(aug_img)
        tensor = final_preprocess(pil_img).unsqueeze(0)
        
        with torch.no_grad():
            logits = classifier(tensor)
            probs = F.softmax(logits, dim=1)[0]
            all_outputs.append(probs)
            
            idx = torch.argmax(probs).item()
            caption = f"{name}\n{class_names[idx]} ({float(probs[idx]):.2f})"
        
        gallery_list.append((aug_img, caption))
            
    avg_probabilities = torch.stack(all_outputs).mean(dim=0)
    return avg_probabilities, gallery_list

# --- STEP 5: INTERFACE ---
with gr.Blocks(title="PepperAI PLC-SN Scanner") as demo:
    gr.Markdown("# 🌶️ PepperAI: SqueezeNet (PLC-SN) Edition")
    gr.Markdown("Using TTA to average results from 16 different views.")
    
    with gr.Row():
        with gr.Column():
            input_view = gr.Image(label="Input Image")
            run_btn = gr.Button("Analyze Leaf (SqueezeNet Pass)", variant="primary")
        with gr.Column():
            res_image = gr.Image(label="Detection result")
            res_label = gr.Label(num_top_classes=2, label="Final Consensus")

    gr.Markdown("### 🔍 Individual Inspection Results")
    res_gallery = gr.Gallery(label="Augmentation Grid", columns=4, rows=4)

    run_btn.click(predict_pepper, input_view, [res_image, res_label, res_gallery])

demo.launch()

/tmp/ipython-input-3397012158.py:27: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  "Gaussian Noise": A.GaussNoise(var_limit=(10, 50), p=1),


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f7c0bba6cb6645adeb.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/protocols/http/httptools_impl.py", line 416, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/applications.py", line 1139, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/applications.py", line 107, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/error


0: 640x640 1 umbrella, 212.6ms
Speed: 4.7ms preprocess, 212.6ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/protocols/http/httptools_impl.py", line 416, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/applications.py", line 1139, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/applications.py", line 107, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/error


0: 640x640 1 umbrella, 212.7ms
Speed: 5.7ms preprocess, 212.7ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/protocols/http/httptools_impl.py", line 416, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/applications.py", line 1139, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/applications.py", line 107, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/error


0: 640x480 (no detections), 163.8ms
Speed: 4.7ms preprocess, 163.8ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 480)


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/protocols/http/httptools_impl.py", line 416, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/applications.py", line 1139, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/applications.py", line 107, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/error


0: 640x480 (no detections), 158.0ms
Speed: 5.1ms preprocess, 158.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 480)


In [ ]:
#resnet18 model 500px

In [ ]:
import cv2
import torch
import torch.nn as nn
import gradio as gr
import albumentations as A
import numpy as np
import os
from ultralytics import YOLO
from torchvision import models, transforms
from PIL import Image
import torch.nn.functional as F

# --- STEP 1: CONFIGURATION ---
MODEL_PATH = '/content/drive/MyDrive/pepper_resnet18_augmented_500px.pth' 

# --- STEP 2: DEFINE THE 16 TTA AUGMENTATIONS ---
tta_transforms = {
    "Original": None,
    "Horizontal Flip": A.HorizontalFlip(p=1),
    "Vertical Flip": A.VerticalFlip(p=1),
    "Rotate 15°": A.Rotate(limit=(15, 15), p=1),
    "Rotate -15°": A.Rotate(limit=(-15, -15), p=1),
    "Brightness +": A.RandomBrightnessContrast(brightness_limit=(0.2, 0.2), contrast_limit=0, p=1),
    "Brightness -": A.RandomBrightnessContrast(brightness_limit=(-0.2, -0.2), contrast_limit=0, p=1),
    "Contrast +": A.RandomBrightnessContrast(brightness_limit=0, contrast_limit=(0.2, 0.2), p=1),
    "Gaussian Noise": A.GaussNoise(var_limit=(10, 50), p=1),
    "Hue Shift": A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=0, val_shift_limit=0, p=1),
    "Saturation +": A.HueSaturationValue(hue_shift_limit=0, sat_shift_limit=30, val_shift_limit=0, p=1),
    "GrayScale": A.ToGray(p=1),
    "Blur": A.Blur(blur_limit=3, p=1),
    "Sharpen": A.Sharpen(p=1),
    "Posterize": A.Posterize(p=1),
    "Solarize": A.Solarize(p=1)
}

final_preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# --- STEP 3: MODEL LOADING ---
def load_pepper_model(path):
    model = models.resnet18(weights=None)
    model.fc = nn.Linear(model.fc.in_features, 2)
    state_dict = torch.load(path, map_location=torch.device('cpu'))
    model.load_state_dict(state_dict)
    model.eval()
    return model

detector = YOLO('yolov8n.pt') 
classifier = load_pepper_model(MODEL_PATH)
class_names = ['HEALTHY', 'UNHEALTHY']

# --- STEP 4: PREDICTION LOGIC ---
def predict_pepper(input_img):
    if input_img is None: return None, None, None
    
    img_rgb = np.array(input_img)
    img_display = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2BGR)
    
    results = detector(img_rgb, conf=0.25)
    detections = results[0].boxes if len(results) > 0 else []
    
    final_probs = torch.zeros(2)
    gallery_data = []

    if len(detections) > 0:
        # Process the first leaf found by YOLO
        box = detections[0]
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        leaf_crop = img_rgb[y1:y2, x1:x2]
        
        # Run TTA on this specific leaf and get results for each view
        final_probs, gallery_data = run_tta_with_individual_labels(leaf_crop)
        
        label_idx = torch.argmax(final_probs).item()
        color = (0, 255, 0) if label_idx == 0 else (0, 0, 255)
        cv2.rectangle(img_display, (x1, y1), (x2, y2), color, 6)
        cv2.putText(img_display, f"Final: {class_names[label_idx]}", (x1, y1-15), 0, 1.2, color, 3)
    else:
        final_probs, gallery_data = run_tta_with_individual_labels(img_rgb)

    confidences = {class_names[i]: float(final_probs[i]) for i in range(2)}
    return cv2.cvtColor(img_display, cv2.COLOR_BGR2RGB), confidences, gallery_data

def run_tta_with_individual_labels(image_np):
    """Predicts each of the 16 images individually to show in the gallery."""
    all_outputs = []
    gallery_list = []
    
    for name, aug in tta_transforms.items():
        # 1. Augment the image
        aug_img = aug(image=image_np)['image'] if aug else image_np
        
        # 2. Prepare for ResNet
        pil_img = Image.fromarray(aug_img)
        tensor = final_preprocess(pil_img).unsqueeze(0)
        
        # 3. Predict INDIVIDUALLY
        with torch.no_grad():
            logits = classifier(tensor)
            probs = F.softmax(logits, dim=1)[0]
            all_outputs.append(probs)
            
            # Identify label for THIS specific augmentation
            idx = torch.argmax(probs).item()
            label_for_this_view = class_names[idx]
            confidence_for_this_view = float(probs[idx])
        
        # 4. Add to gallery with the technique name + its specific result
        caption = f"{name}\nResult: {label_for_this_view} ({confidence_for_this_view:.2f})"
        gallery_list.append((aug_img, caption))
            
    avg_probabilities = torch.stack(all_outputs).mean(dim=0)
    return avg_probabilities, gallery_list

# --- STEP 5: GRADIO LAYOUT ---
with gr.Blocks(title="PepperAI Deep Inspection") as demo:
    gr.Markdown("# 🌶️ PepperAI: Deep Inspection Mode")
    gr.Markdown("Every one of the 16 views below is analyzed independently by ResNet-18 before averaging.")
    
    with gr.Row():
        with gr.Column():
            input_view = gr.Image(label="Input Image")
            run_btn = gr.Button("Analyze Leaf (Full 16-Image Pass)", variant="primary")
        with gr.Column():
            res_image = gr.Image(label="Detection result")
            res_label = gr.Label(num_top_classes=2, label="Final Consensus (TTA Averaged)")

    gr.Markdown("### 🔍 Individual Inspection Results (The 16 Views)")
    res_gallery = gr.Gallery(label="Augmentation Prediction Grid", columns=4, rows=4)

    run_btn.click(predict_pepper, input_view, [res_image, res_label, res_gallery])

demo.launch()

/tmp/ipython-input-4184742049.py:26: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  "Gaussian Noise": A.GaussNoise(var_limit=(10, 50), p=1),


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://80b213e217d6e2712e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/protocols/http/httptools_impl.py", line 416, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/applications.py", line 1139, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/applications.py", line 107, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/error


0: 640x640 1 umbrella, 194.4ms
Speed: 6.1ms preprocess, 194.4ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/protocols/http/httptools_impl.py", line 416, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/applications.py", line 1139, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/applications.py", line 107, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/error


0: 640x480 (no detections), 283.4ms
Speed: 7.7ms preprocess, 283.4ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 (no detections), 179.2ms
Speed: 8.6ms preprocess, 179.2ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 (no detections), 192.9ms
Speed: 7.1ms preprocess, 192.9ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 480)


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/protocols/http/httptools_impl.py", line 416, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/applications.py", line 1139, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/applications.py", line 107, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/error


0: 640x480 (no detections), 316.3ms
Speed: 11.1ms preprocess, 316.3ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 (no detections), 175.5ms
Speed: 6.5ms preprocess, 175.5ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 1 fire hydrant, 1 vase, 198.0ms
Speed: 9.7ms preprocess, 198.0ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 1 fire hydrant, 1 vase, 155.6ms
Speed: 7.3ms preprocess, 155.6ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 1 fire hydrant, 1 vase, 287.8ms
Speed: 14.4ms preprocess, 287.8ms inference, 2.1ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 1 fire hydrant, 1 vase, 158.5ms
Speed: 4.9ms preprocess, 158.5ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 480)


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/protocols/http/httptools_impl.py", line 416, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/applications.py", line 1139, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/applications.py", line 107, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/error


0: 640x480 1 fire hydrant, 1 vase, 167.4ms
Speed: 6.5ms preprocess, 167.4ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 1 fire hydrant, 1 vase, 178.5ms
Speed: 5.0ms preprocess, 178.5ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 1 fire hydrant, 1 vase, 250.1ms
Speed: 10.0ms preprocess, 250.1ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 480)

0: 448x640 (no detections), 167.0ms
Speed: 4.6ms preprocess, 167.0ms inference, 1.0ms postprocess per image at shape (1, 3, 448, 640)


In [ ]:
#resnet18 model 224px
#MODEL_PATH = '/content/drive/MyDrive/resnet18-f37072fd.pth' 


In [27]:
import cv2
import torch
import torch.nn as nn
import gradio as gr
import albumentations as A
import numpy as np
import os
from ultralytics import YOLO
from torchvision import models, transforms
from PIL import Image
import torch.nn.functional as F

# --- STEP 1: CONFIGURATION ---
# IMPORTANT: Ensure your Drive is mounted: 
# from google.colab import drive; drive.mount('/content/drive')
MODEL_PATH = '/content/drive/MyDrive/resnet18-f37072fd.pth' 
YOLO_PATH = 'yolov8n.pt' 

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# --- STEP 2: LOAD MODELS SAFELY ---
def load_models():
    # 1. Load YOLO
    try:
        print("Loading YOLO...")
        det = YOLO(YOLO_PATH)
    except Exception as e:
        print(f"YOLO Load Error: {e}")
        det = None

    # 2. Load ResNet
    print("Loading ResNet...")
    res = models.resnet18(weights=None)
    res.fc = nn.Linear(res.fc.in_features, 2) # Force 2 classes
    
    if os.path.exists(MODEL_PATH):
        try:
            sd = torch.load(MODEL_PATH, map_location=device)
            res.load_state_dict(sd, strict=False)
            print("ResNet weights loaded successfully.")
        except Exception as e:
            print(f"Weight loading error (Check if your .pth is actually a ResNet18): {e}")
    else:
        print(f"❌ MODEL FILE NOT FOUND AT: {MODEL_PATH}")
    
    return det, res.to(device).eval()

detector, classifier = load_models()
class_names = ['HEALTHY', 'UNHEALTHY']

# Preprocessing
final_preprocess = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# --- STEP 3: LOGIC ---
def predict_pepper(input_img):
    if input_img is None: return None, None, []
    
    try:
        # Convert to BGR for OpenCV
        img_bgr = cv2.cvtColor(input_img, cv2.COLOR_RGB2BGR)
        
        # Detection
        results = detector(input_img, conf=0.25, verbose=False)
        boxes = results[0].boxes.xyxy.cpu().numpy() if len(results) > 0 else []
        
        if len(boxes) > 0:
            x1, y1, x2, y2 = map(int, boxes[0])
            crop = input_img[y1:y2, x1:x2]
            if crop.size == 0: crop = input_img # Backup if crop fails
        else:
            crop = input_img
            x1, y1, x2, y2 = 0, 0, 0, 0

        # TTA Pass
        all_probs = []
        gallery = []
        
        # Simpler TTA for debugging
        augs = [None, A.HorizontalFlip(p=1), A.VerticalFlip(p=1), A.RandomBrightnessContrast(p=1)]
        names = ["Original", "H-Flip", "V-Flip", "Bright"]
        
        for name, aug in zip(names, augs):
            aug_img = aug(image=crop)['image'] if aug else crop
            tensor = final_preprocess(aug_img).unsqueeze(0).to(device)
            
            with torch.no_grad():
                output = classifier(tensor)
                prob = F.softmax(output, dim=1)[0].cpu()
                all_probs.append(prob)
                idx = torch.argmax(prob).item()
                gallery.append((aug_img, f"{name}: {class_names[idx]}"))

        avg_prob = torch.stack(all_probs).mean(0)
        final_idx = torch.argmax(avg_prob).item()
        
        # Draw on image
        color = (0, 255, 0) if final_idx == 0 else (255, 0, 0)
        if len(boxes) > 0:
            cv2.rectangle(img_bgr, (x1, y1), (x2, y2), color, 4)
        
        res_img = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
        confidences = {class_names[i]: float(avg_prob[i]) for i in range(2)}
        
        return res_img, confidences, gallery

    except Exception as e:
        print(f"CRITICAL RUNTIME ERROR: {e}")
        return input_img, {"Error": 1.0}, []

# --- STEP 4: UI ---
with gr.Blocks() as demo:
    gr.Markdown("# 🌶️ PepperAI Diagnostic Tool")
    with gr.Row():
        with gr.Column():
            in_img = gr.Image(label="Upload Pepper Leaf")
            btn = gr.Button("Analyze", variant="primary")
        with gr.Column():
            out_img = gr.Image(label="Result")
            out_label = gr.Label(label="Consensus")
    
    out_gal = gr.Gallery(label="TTA Views", columns=4)
    
    btn.click(predict_pepper, inputs=in_img, outputs=[out_img, out_label, out_gal])

demo.launch(debug=True)

Using device: cpu
Loading YOLO...
Loading ResNet...
Weight loading error (Check if your .pth is actually a ResNet18): Error(s) in loading state_dict for ResNet:
	size mismatch for fc.weight: copying a param with shape torch.Size([1000, 512]) from checkpoint, the shape in current model is torch.Size([2, 512]).
	size mismatch for fc.bias: copying a param with shape torch.Size([1000]) from checkpoint, the shape in current model is torch.Size([2]).
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://f7e5f7fa166f6db9c2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory 

ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/protocols/http/httptools_impl.py", line 416, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/applications.py", line 1139, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/applications.py", line 107, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/error

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7870 <> https://f7e5f7fa166f6db9c2.gradio.live


In [ ]:
#vit (vision transformer) model

In [12]:
import torch
import cv2
import numpy as np
import albumentations as A
from albumentations.pytorch import ToTensorV2
from ultralytics import YOLO
import timm
import gradio as gr
from PIL import Image

# --- 1. SETUP & LOAD MODELS ---
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# USE YOUR FINE-TUNED WEIGHTS HERE
yolo_path = '/content/drive/MyDrive/best.pt'
yolo_model = YOLO(yolo_path) 

# Load your best ViT classification model
vit_model = timm.create_model('vit_base_patch16_384', pretrained=False, num_classes=2)
vit_model.load_state_dict(torch.load('/content/drive/MyDrive/pepperai_best_model.pth', map_location=DEVICE))
vit_model.to(DEVICE).eval()

# ViT Output Classes
CLASSES = ['NOT FUNGAL INFECTED', 'FUNGAL INFECTED']

# --- 2. THE 16 SPECIFIC AUGMENTATIONS (Fixed Syntax) ---
# We use the specific list from your project configuration
augmentations_list = [
    ("Original", A.NoOp()),
    ("Brightness", A.RandomBrightnessContrast(brightness_limit=(0.2, 0.2), contrast_limit=0, p=1)),
    ("Contrast", A.RandomBrightnessContrast(brightness_limit=0, contrast_limit=(0.2, 0.2), p=1)),
    ("Random Crop", A.RandomResizedCrop(size=(384, 384), scale=(0.8, 0.8), p=1)),
    ("Horizontal Flip", A.HorizontalFlip(p=1)),
    ("Vertical Flip", A.VerticalFlip(p=1)),
    ("Gaussian Noise", A.GaussNoise(var_limit=(10.0, 50.0), p=1)),
    ("Poisson Noise", A.ISONoise(color_shift=(0.01, 0.05), intensity=(0.1, 0.5), p=1)),
    ("Rotation 15", A.Rotate(limit=(15, 15), p=1)),
    ("Saturation", A.HueSaturationValue(hue_shift_limit=0, sat_shift_limit=(30, 30), val_shift_limit=0, p=1)),
    ("Jitter", A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1, p=1)),
    ("Hist Equalized", A.Equalize(p=1)),
    ("Translated", A.Affine(translate_percent={"x": 0.1, "y": 0.1}, p=1)),
    ("Unsharp", A.UnsharpMask(p=1)),
    ("Blur", A.GaussianBlur(blur_limit=(3, 7), p=1)),
    ("GrayScale", A.ToGray(p=1))
]

def process_manual_test(input_img):
    # --- STEP 1: YOLO DETECTION ---
    # We lower the confidence to 0.15 to handle watermarks/logos
    results = yolo_model(input_img, conf=0.15, verbose=False)
    
    # Check for any of your 4 trained classes: damaged, fungus, healthy, sick
    if not results[0].boxes:
        return input_img, [], "⚠️ No leaf detected by the fine-tuned model."

    # Draw box on original image for display
    img_with_box = input_img.copy()
    box = results[0].boxes.xyxy[0].cpu().numpy().astype(int)
    cv2.rectangle(img_with_box, (box[0], box[1]), (box[2], box[3]), (0, 255, 0), 8)
    
    # Crop and Resize leaf for ViT
    leaf_crop = input_img[box[1]:box[3], box[0]:box[2]]
    leaf_crop = cv2.resize(leaf_crop, (384, 384))

    # --- STEP 2: APPLY 16 AUGMENTATIONS & VOTING ---
    aug_gallery = []
    total_scores = np.zeros(2)

    for name, aug in augmentations_list:
        aug_pipeline = A.Compose([aug, A.Normalize(), ToTensorV2()])
        transformed = aug_pipeline(image=leaf_crop)["image"].unsqueeze(0).to(DEVICE)
        
        with torch.no_grad():
            output = torch.nn.functional.softmax(vit_model(transformed), dim=1)
            scores = output.cpu().numpy()[0]
            
        label = CLASSES[np.argmax(scores)]
        conf = np.max(scores) * 100
        total_scores += scores
        
        # Build gallery item
        aug_gallery.append((leaf_crop, f"{name}: {label} ({conf:.1f}%)"))

    # Final Average Logic
    avg_scores = total_scores / 16
    final_idx = np.argmax(avg_scores)
    final_label = CLASSES[final_idx]
    final_conf = avg_scores[final_idx] * 100
    
    return img_with_box, aug_gallery, f"✅ Final Verdict: {final_label} ({final_conf:.2f}% Confidence)"

# --- 3. GRADIO LAYOUT ---
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🌶️ PepperAI: Advanced Leaf Diagnostics")
    gr.Markdown("Detecting leaves with fine-tuned YOLO and classifying with 16-view ViT voting.")
    
    with gr.Row():
        with gr.Column():
            input_view = gr.Image(label="Step 1: Upload Photo")
            btn = gr.Button("🔍 Analyze Leaf", variant="primary")
        with gr.Column():
            output_box = gr.Image(label="Step 2: Detection Result")
            verdict = gr.Textbox(label="Step 3: Final Result", interactive=False)
    
    gallery = gr.Gallery(label="Detailed Analysis (All 16 Augmentations)", columns=4, rows=4)
    
    btn.click(process_manual_test, inputs=input_view, outputs=[output_box, gallery, verdict])

demo.launch(share=True)

/tmp/ipython-input-2555398275.py:35: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  ("Gaussian Noise", A.GaussNoise(var_limit=(10.0, 50.0), p=1)),
/tmp/ipython-input-2555398275.py:93: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=gr.themes.Soft()) as demo:


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://de3d6bb0ab422fb96a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/protocols/http/httptools_impl.py", line 416, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/applications.py", line 1139, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/applications.py", line 107, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/error